In [ ]:
import os
import functools

import cv2
import numpy as np

from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise


In [ ]:
def trajectory_generator(env, policy, act_noise_pct, res=(640, 480)):
    action_space_ptp = env.action_space.high - env.action_space.low

    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        yield r, done, info, env.sim.render(*res, mode='offscreen', camera_name='topview')[:,:,::-1]

def writer_for(tag, fps, res):
    if not os.path.exists('movies'):
        os.mkdir('movies')
    return cv2.VideoWriter(
        f'movies/{tag}.avi',
        cv2.VideoWriter_fourcc('M','J','P','G'),
        fps,
        res
    )

In [ ]:
resolution = (1920, 1080)

config = [
    # env, action noise pct, cycles, quit on success
    ['button-press-topdown-v2', np.zeros(4), 3, False],
    ['pick-place-v2', np.zeros(4), 3, False],
    ['reach-v2', np.zeros(4), 3, False],
    ['window-open-v2', np.zeros(4), 3, False],
    ['sweep-v2', np.zeros(4), 3, False],
    ['sweep-into-v2', np.zeros(4), 3, False],
    ['shelf-place-v2', np.zeros(4), 3, False],
    ['push-v2', np.zeros(4), 3, False],
    ['peg-insert-side-v2', np.zeros(4), 3, False],
    ['lever-pull-v2', np.zeros(4), 3, False],
]

for env, noise, cycles, quit_on_success in config:
    tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)

    policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
    env = ALL_ENVS[env]()
    env._partially_observable = False
    env._freeze_rand_vec = False
    env._set_task_called = True

    writer = writer_for(tag, env.metadata['video.frames_per_second'], resolution)
    for _ in range(cycles):
        for r, done, info, img in trajectory_generator(env, policy, noise, resolution):
            writer.write(cv2.rotate(img, cv2.ROTATE_180))
            if quit_on_success:
                break

    writer.release()